# CO2e Scoring Notebook (Using EPA Mapping)

Last Updated November 29, 2023

In [ ]:
import os
import requests
import json
import ast
import pandas as pd
import yaml

### Retrieve the Credential

In [ ]:
credentials_file = '../../config/credentials/wml-credentials.json'

with open(credentials_file, "r") as f:
    credentials = json.load(f)

In [ ]:
# Assuming you have these variables set somewhere in your script
USERNAME = credentials["username"]
API_KEY = credentials["apikey"]
DATAPLATFORM_URL = credentials["url"]

In [ ]:
headers = {
    'Content-type': 'application/json'
}
data = {
    'username': USERNAME,
    'api_key': API_KEY
}

In [ ]:
response = requests.post(
    f"{DATAPLATFORM_URL}/icp4d-api/v1/authorize", 
    headers=headers, 
    data=json.dumps(data),
    verify=False  # This corresponds to the '-k' or '--insecure' flag in curl
)
token = response.text
dict_token = ast.literal_eval(token)
bearer_token = dict_token['token']

### Load the Data

In [ ]:
df = pd.read_csv('../data/co2e_scoring.csv')

In [ ]:
df.tail(10)

In [ ]:
df_loss = df[[ 'energy_loss','location_zip']]

In [ ]:
# df_loss = df_loss.dropna()
df_loss = df_loss.reset_index(drop = True)

In [ ]:
df_loss.columns

In [ ]:
# df_loss['location_zip_list'] = df_loss['location_zip']

In [ ]:
location_zip = '99557'
df_loss['location_zip'] = location_zip

In [ ]:
df_loss.shape

### Send the Data for Scoring

In [ ]:
model_factory_config_file_name = "../config/deployment_info.yml"

with open(model_factory_config_file_name, 'r') as file:
    model_factory_config = yaml.safe_load(file)

print(model_factory_config)

In [ ]:
# retrieve the CP4D endpoint at the IBM Watson Studio
scoring_url = model_factory_config['cp4d_url']
scoring_url

In [ ]:
#array_of_input_fields = list(df.columns)
array_of_input_fields=['energy_loss', 'location_zip']

array_of_values_to_be_scored = df_loss[array_of_input_fields].values.tolist()

In [ ]:
%%time
import urllib3, requests, json

#scoring_url = 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/f1ec7cd7-6db7-4e52-a958-ce1beed57e49/predictions?version=2023-11-08'

# NOTE: you must construct mltoken based on provided documentation
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + bearer_token}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"input_data": [{"fields": array_of_input_fields, "values": array_of_values_to_be_scored}]}
# print("Input payload")
# print(json.dumps(payload_scoring, indent=2))

response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header, verify=False)
print("Scoring response")
predictions = json.loads(response_scoring.text)
print(json.dumps(predictions, indent=2))

In [ ]:
predictions

### Combine the Results

In [ ]:
output = predictions["predictions"][0]["values"]

In [ ]:
reshaped_data = [row[0] for row in output]
column_names = ['CO2 (g)']

predictions_df = pd.DataFrame(reshaped_data, columns=column_names)
display(predictions_df)

In [ ]:
df.shape

In [ ]:
df_merged = pd.concat([df, predictions_df], axis=1)

In [ ]:
df_merged.tail(10)